In [ ]:
import geemap
import ee

Map = geemap.Map(center=[30, 112])
Map

In [ ]:
roi_file = r'E:/Paper/Data/roi_shp/jiujiang.shp'
# point_file = r'E:/Paper/Data/landuse/points.shp'

In [ ]:
roi = geemap.shp_to_ee(roi_file)
psv = ee.FeatureCollection('users/2431566134liumonarch/points')

In [ ]:
Map.addLayer(roi, {}, 'roi')
Map.addLayer(psv, {'color': 'FF0000'}, 'ps')

In [ ]:
from Monarch import user_gee
year = 2019
s2_year = []
for i in range(3, 12):
    s2 = ee.ImageCollection("COPERNICUS/S2")\
            .filterBounds(roi)\
            .sort('CLOUD_COVERAGE_ASSESSMENT', False)\
            .filterDate(str(year), str(year+1))\
            .filter(ee.Filter.calendarRange(i, i, 'month'))
    s2_rm_cloud = s2.map(user_gee.rm_cloud_s2_sr)
    s2_z = s2.merge(s2_rm_cloud).mosaic()
    a = s2.aggregate_array('system:time_start').map(lambda i: ee.Date(i).format('YYYY-MM-dd'))
    s2_image = s2_z.addBands(s2_z.normalizedDifference(['B8', 'B11']).rename("S2_LSWI"))
    s2_image = s2_image.addBands(s2_z.normalizedDifference(['B8', 'B4']).rename("S2_NDVI_B84"))
    s2_year.append(s2_image)

In [ ]:
image = ee.ImageCollection(s2_year).toBands().clip(roi.geometry())

In [ ]:
ps = psv.filter(ee.Filter.lte('CID', 70))
ps_y = psv.filter(ee.Filter.gt('CID', 70))

In [ ]:
label = 'RASTERVALU'

training = image.sampleRegions(**{
  'collection': psv,
  'properties': [label],
  'scale': 10
})
# 使用Cart(分类与回归树)
trained = ee.Classifier.smileCart().train(training, label)

In [ ]:
result = image.classify(trained)

Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map